In [1]:
!pip install pycaret[full]
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 808.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of selenium to determine which version is compatible with other requirements. This

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"riship3499","key":"f05439e4bf2d33cd99322d4f479bfb47"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!chmod 400 ~/.kaggle/kaggle.json

In [31]:
!kaggle datasets download -d robikscube/hourly-energy-consumption
#!unzip store-sales-time-series-forecasting.zip

Dataset URL: https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption
License(s): CC0-1.0
 44% 5.00M/11.4M [00:00<00:00, 50.7MB/s]
100% 11.4M/11.4M [00:00<00:00, 94.5MB/s]


In [32]:
!unzip hourly-energy-consumption.zip

Archive:  hourly-energy-consumption.zip
  inflating: AEP_hourly.csv          
  inflating: COMED_hourly.csv        
  inflating: DAYTON_hourly.csv       
  inflating: DEOK_hourly.csv         
  inflating: DOM_hourly.csv          
  inflating: DUQ_hourly.csv          
  inflating: EKPC_hourly.csv         
  inflating: FE_hourly.csv           
  inflating: NI_hourly.csv           
  inflating: PJME_hourly.csv         
  inflating: PJMW_hourly.csv         
  inflating: PJM_Load_hourly.csv     
  inflating: est_hourly.paruqet      
  inflating: pjm_hourly_est.csv      


In [40]:
# Step 3: Load the dataset and preprocess it
import pandas as pd
import zipfile

# Unzip the downloaded file
with zipfile.ZipFile('hourly-energy-consumption.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Load the dataset
data = pd.read_csv('AEP_hourly.csv')



# Convert 'Datetime' to datetime and set it as index
data['Datetime'] = pd.to_datetime(data['Datetime'])
data.set_index('Datetime', inplace=True)






In [41]:
import numpy as np

# Creating a mock 'temperature' exogenous variable
data['temperature'] = 20 + 10 * np.sin(np.linspace(0, 10, len(data)))

# Resample the data to daily frequency (if needed)
data = data.resample('D').mean()

# Fill any missing values
data.fillna(method='ffill', inplace=True)

# Step 5: Check if the frequency is set correctly
print(data.index.freq)  # Should print <Day>

<Day>


In [42]:
# Step 6: Reassign target and exogenous variables after fixing the frequency
target = data[['AEP_MW']]  # Electricity load (target variable)
exogenous = data[['temperature']]  # Temperature as the exogenous variable

# Step 7: Setup PyCaret for time series forecasting
from pycaret.time_series import *

# Initialize the setup
ts_setup = setup(data=target,
                 target='AEP_MW',
                 fh=30,  # Forecast horizon (e.g., next 30 days)
                 fold=3,  # Cross-validation fold
                 session_id=123,
                 seasonal_period='D')  # Daily seasonality

,Description,Value
0,session_id,123
1,Target,AEP_MW
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(5055, 1)"
5,Transformed data shape,"(5055, 1)"
6,Transformed train set shape,"(5025, 1)"
7,Transformed test set shape,"(30, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [50]:
# Step 9: Finalize the model
final_model = finalize_model('prophet')

future_dates = pd.date_range(start=data.index[-1] + pd.DateOffset(days=1), periods=30, freq='D')
future_exogenous = pd.DataFrame({'temperature': 20 + 10 * np.sin(np.linspace(0, 10, 30))}, index=future_dates)
# Make predictions using the future exogenous variables
future_forecast = predict_model(final_model, X=future_exogenous)
future_forecast.head()

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf3sv8ei9/nky67pkv.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpf3sv8ei9/a_9cj3eu.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=66395', 'data', 'file=/tmp/tmpf3sv8ei9/nky67pkv.json', 'init=/tmp/tmpf3sv8ei9/a_9cj3eu.json', 'output', 'file=/tmp/tmpf3sv8ei9/prophet_modelzxbdroak/prophet_model-20240920063513.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


,y_pred
2018-08-04,15176.0255
2018-08-05,14734.2616
2018-08-06,16314.8166
2018-08-07,16533.7163
2018-08-08,16464.4285


In [51]:
plot_model(final_model, plot='forecast')